In [1]:
from new_session import *
start_session()

FloatProgress(value=0.0, max=7200.0)

Session started, will remain active for two hours as counted by the progress bar


<h1>Motion of Charged Objects in Electric and Magnetic Fields</h1>

In this lab, you will 

<ol>
    <li> Calculate the net force acting on a charged object due to both E and B fields.</li>
    <li> Update the velocity and position of the object based on the force acting on it using Newton's 2nd law.</li>
    <li> Run a visualization of a charged particle moving through uniform B and E fields.</li>
    <li> Run a visualization of the cyclotron motion.</li>
</ol>



<h2>Task 1: Calculate the Lorentz Force on a Charged Particle & Track its Velocity and Position</h2>

When a charged object is within a region of Electric and/or Magnetic fields, the net force on it can be calculated as:

$$\vec{F} = q\vec{E} + q\vec{v} \times \vec{B}$$

This net electromagnetic force is called the Lorentz force.

To find the trajectory of the charged object within the field(s), we need to utilize Newton's 2nd law to examine the effect of the Lorentz force on the momentum (and hence velocity) of the object.

$$\vec{p_f} = \vec{p_i} + \int_{t_i}^{t_f} \vec{F} dt$$

One way to build a simulation that tracks the trajectory of the system is to divide the time interval $(t_f - t_i)$ into small time-steps $dt$$ such that the we can treat the force as a constant during each short time interval. Then we can update the velocity and position vector after each time-step as follows:

$$\vec{v}(t+dt) = \vec{v}(t) + \frac{\vec{F} dt}{m}$$

$$\vec{r}(t+dt) = \vec{r}(t) + \vec{v}(t) dt + \frac{1}{2} \frac{\vec{F}}{m} (dt)^2$$ [Note that the last term is of order $(dt)^2$ and hence can be ignored if $dt$ is small enough]

<b>In this task, you will complete three functions that will be used to calculate the trajectory of a charged object</b>:

1- <i>Lorentz_F(q,v,E,B)</i>: Returns the Lorentz force on a charged object

2- <i>update_v(vi,m,F,dt)</i>: Returns $\vec{v}(t+dt)$ given $\vec{v}(t)$

3- <i>update_position(ri,vi,m,F,dt)</i>: Returns $\vec{r}(t+dt)$ given $\vec{r}(t)$ and $\vec{v}(t)$

Each function is missing only one line of code where you will implement the equations provided above.

In [2]:
from vpython import *


def Lorentz_F(q,v,E,B):
    # calculates and returns the Lorentz force given:
    # q= charge of object, v= velocity vector, E= Electric field vector and B= Magnetic field vector
    # In vpython, cross(A,B) returns the cross product A x B
    F = q*(E+cross(v,B))
    return F
def update_v(vi,m,F,dt):
    # calculates and returns the final velocity vector after a short time-step, i.e. v(t+dt) given:
    # vi = initial velocity v(t), m = mass, F = net force on the object and dt = time step
    vf = vi + F*dt/m
    return vf
def update_position(ri,vi,m,F,dt):
    # calculates and returns the final position vector after a short time-step, i.e. r(t+dt) given:
    # ri = initial position v(t), vi = initial velocity v(t), m = mass, F = net force on the object and dt = time step
    rf = ri + vi*dt + 0.5*F*dt*dt/m
    return rf


<IPython.core.display.Javascript object>

<5e-09, -2e-09, 0>

<h2>Task 2: Track The Motion of a Charged Object Within Uniform E and B Fields</h2>

We will now use the functions you built to track the motion of a charged object in uniform fields. 

We will focus on a scenario where a charged object with charge $q$ and mass $m$ initially moving in the x direction with some velocity $v_x$ goes through a region of uniform vertical E field $E_y$ and/or a uniform B field in the z-direction $B_z$. Note that the quantities $q, v_x, E_y, B_z$ could be positive, zero or negative.

In [38]:
import math
def simulate_motion(q=0,m=1,vx=1,Ey=0,Bz=0,total_time=20):
    print("White arrows = E field, Yellow arrows = B field, Blue = negative charge and Red = positive charge")
    print("E and B fields are assumed to be uniform all over space.")
    if q!=0:
        scale = math.floor(math.log10(abs((q*vx*Bz+q*Ey)/m)))
        q = q*10**(-scale)
    scene = canvas() 
    ######################################################
    #define parameters for the moving point charge
    obj = sphere(radius = 1) 
    obj.charge = q
    obj.mass = m
    obj.pos = vector(0,0,0) #initial position vector
    obj.velocity = vector(vx,0,0) #initial velocity vector
    obj.color = color.blue
    if obj.charge >0:
        obj.color = color.red
    ######################################################
    #define the uniform E and B fields 
    E = vector(0,Ey,0)
    B = vector(0,0,Bz)
    ######################################################
    t = 0 #initial time
    dt = 0.001 #timestep to evolve the system
    trail = curve(color=color.white)
    max_y = 15
    max_x = 50
    if E.y != 0:
        mybox = box(pos=vector(0,max_y*E.y/abs(E.y),0), length=max_x, height=0.1, width=50,color=color.blue)
        mybox = box(pos=vector(0,-max_y*E.y/abs(E.y),0), length=max_x, height=0.1, width=50,color=color.red)
        dx = max_x/10
        for x in arange(-max_x/2, max_x/2+dx, 2*dx):
            arrow(pos=vector(x,-max_y*E.y/abs(E.y),0),axis = vector(0,2*max_y*E.y/abs(E.y),0),color=color.white,shaftwidth=0.1)
        scene.range = max_y*2*1.1
        dx = max_x/10
    else:
        max_x = 10
        max_y = 8
        dx = max_x/4
        obj.radius = 0.6
    if B.z != 0:

        
        for x in arange(-max_x/2, max_x/2+dx, 2*dx):
            arrow(pos=vector(x,-max_y/4,0),axis = vector(0,0,2*max_y*B.z/abs(B.z)),color=color.yellow,shaftwidth=0.1)
            arrow(pos=vector(x,max_y/4,0),axis = vector(0,0,2*max_y*B.z/abs(B.z)),color=color.yellow,shaftwidth=0.1)
        #scene.center = vector(max_x/3,0,0)


    
    final_time = total_time
    while t<final_time:

        rate(600) #slow down the rate of visualization to 600 updates per sec
        F = Lorentz_F(obj.charge,obj.velocity,E,B) #calculate the net force on the charge
        r = update_position(obj.pos,obj.velocity,obj.mass,F,dt)
        v = update_v(obj.velocity,obj.mass,F,dt)
        obj.pos = r
        obj.velocity = v
        trail.append(pos=r)
        t = t+dt
        if abs(r.y) >=max_y:
            break


In [51]:
simulate_motion(q=10e-9,m=10e-31,vx=0,Ey=1,Bz=1,total_time = 20)

White arrows = E field, Yellow arrows = B field, Blue = negative charge and Red = positive charge
E and B fields are assumed to be uniform all over space.


<IPython.core.display.Javascript object>

In [41]:
scene = canvas() 
trail = curve(color=color.yellow)
myell = ellipsoid(pos=vector(0,0,0),length=1, height=1, width=0.1,color=color.white,opacity = 0.3)
box(pos=vector(0,0,0), length=0.2, height=1, width=0.1,color=color.black,opacity = 0.8)
scene.lights = []
distant_light(direction=vec( 1.1,  0.44,  0.88), color=color.gray(0.8))
obj = sphere(radius = 0.06) 
obj.charge = 1
obj.mass = 1 
obj.pos = vector(0,0,0) #initial position vector
obj.velocity = vector(0,0,0) #initial velocity vector
obj.color = color.blue
if obj.charge >0:
    obj.color = color.red
    

######################################################
t = 0 #initial time
dt = 0.001 #timestep to evolve the system
in_B = False
n = 1
while obj.pos.mag <0.5:
    
    rate(600) #slow down the rate of visualization to 600 updates per sec
    if abs(obj.pos.x) >=0.1:
        E = vector(0,0,0)
        B = vector(0,0,4)
        in_B = True
    else:
        if in_B:
            in_B = False
            n = -n
        E = vector(0.3*n,0,0)
        B = vector(0,0,0)
        
    F = Lorentz_F(obj.charge,obj.velocity,E,B) #calculate the net force on the charge
    r = update_position(obj.pos,obj.velocity,obj.mass,F,dt)
    v = update_v(obj.velocity,obj.mass,F,dt)
    obj.pos = r
    obj.velocity = v
    trail.append(pos=r)
    t = t+dt
    
    

<IPython.core.display.Javascript object>